![alt text](FGV_logo.png)

# Machine Learning

* Link para os modulos de machine learning: [SciKit Learn](http://scikit-learn.org/stable/)
* Curso Machine Learning Andrew Ng: [Coursera](https://www.coursera.org/learn/machine-learning)
* Curso Data Analysis Jose Portilla: [Udemy](https://www.udemy.com/learning-python-for-data-analysis-and-visualization/learn/v4/t/lecture/2338236?start=0)
* Curso CS109 Harvard: [Harvard](http://cs109.github.io/2015)

# Regressão

***

***
<div class="span5 alert alert-info">

<p> Fornecidos $X$ and $Y$, o objetivo da regressão linear é: </p>
<ul>
  <li> Criar um <b>modelo preditivo</b> para predizer o $Y$ a partir de $X_i$ futuros </li>
  <li> Modelar a <b>importancia</b> entre cada variável dependente $X_i$ e $Y$</li>
    <ul>
      <li> Nem todos os $X_i$ tem relação com $Y$</li>
      <li> Quais $X_i$ que mais contribuem para determinar $Y$? </li>
    </ul>
</ul>
</div>

### recap
***

[Regressão Linear](http://en.wikipedia.org/wiki/Linear_regression) é um metodo para modelar a relação entre um conjunto de variaveis independentes $X$ (explanatórias, features, preditores) e uma variável dependente $Y$.  Esse metodo assume que $X$ tem uma relação linear com $Y$.  

$$ Y = \beta_0 + \beta_1 X + \epsilon$$

one $\epsilon$ refere-se a um erro. 

* $\beta_0$ é a intercepto do modelo

* Regressão Linear Multipla é quando há mais de uma variavel independente
    * $X_1$, $X_2$, $X_3$, $\ldots$

$$ Y = \beta_0 + \beta_1 X_1 + \ldots + \beta_p X_p + \epsilon$$ 

* O objetivo será estimar os coeficientes (e.g. $\beta_0$ and $\beta_1$). Representamos as estimativas com o "chapeu" ao topo da letra. 

$$ \hat{\beta}_0, \hat{\beta}_1 $$

* Uma vez obtido a estimativa dos coeficientes $\hat{\beta}_0$ and $\hat{\beta}_1$, podemos usar para predizer novos valores de $Y$

$$\hat{y} = \hat{\beta}_0 + \hat{\beta}_1 x_1$$

In [ ]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

#### Importando o dataset:

In [ ]:
boston = load_boston()

In [ ]:
print(boston.DESCR)

#### Carregando o dataframe

In [ ]:
# carregando o df
boston_df = pd.DataFrame(boston.data)

# nome das colunas
boston_df.columns = boston.feature_names

#### Explorando o dataframe

In [ ]:
boston_df.head()

In [ ]:
boston_df.plot(kind='scatter', x = 'RM', y = 'AGE')

In [ ]:
sns.jointplot(data=boston_df, x = 'RM', y = 'AGE', kind = 'hex')

In [ ]:
sns.jointplot(data=boston_df, x = 'RM', y = 'AGE', kind = 'kde')

In [ ]:
# introduzindo a coluna de precos
boston_df['Preco'] = boston.target

In [ ]:
boston_df.head()

In [ ]:
# Histograma dos preços (alvo da predição)
plt.hist(boston_df['Preco'],bins=50)

# Nome dos eixos
plt.xlabel('Precos in $1000s')
plt.ylabel('Numero de towns')

In [ ]:
boston_df_sample = boston_df.sample(frac = 0.1)

In [ ]:
# Plotando a coluna #5 (RM)
# plt.scatter(boston_df['RM'], boston_df['Preco'])

sns.jointplot(data = boston_df, x = 'RM', y = 'Preco', kind = 'hex')

#label
plt.ylabel('Precos em $1000s')
plt.xlabel('Media da qtd de comodos por habitacao')

### Problema de negócio: Quero predizer o preço.

#### Se eu tivesse somente uma feature...
(e usando scipy)

In [ ]:
# como seria...
sns.lmplot('RM', 'Preco', data=boston_df, fit_reg=True)

In [ ]:
# Tentem isso tambem...
# sns.jointplot('RM', 'Preco', data=boston_df, kind = 'reg')

#### Objetivo: encontrar os "melhores" $a$ e $b$ 

$y = a.x + b$

onde

* $y$ : preço
* $x$ : qtd média de quartos

In [ ]:
from scipy import stats
import numpy as np
X = boston_df.RM
y = boston_df.Preco
a, b, r_value, p_value, std_err = stats.linregress(X,y)

inclinacao, intercepto = a, b  # 

In [ ]:
print (inclinacao)
print (intercepto)

In [ ]:
quartos = np.array(X)
precos = np.array(y)

pred = a * quartos + b 


In [ ]:
# RMSE
rmse = np.sqrt(np.mean((pred - precos) ** 2))
print ('RMSE =', rmse)

Uma interpretação do RMSE

In [ ]:
r = 6
p = a * r + b

print ('Para uma cidade (town) cuja media de comodos é', r, 'comodos...')
print ('o preço previsto será %.2f, e ...'% p)
print ('... em 68%% das observações, o preco fica entre %.2f e %.2f.' % (p - rmse, p + rmse))
print ('... em 95%% das observações, o preco fica entre %.2f e %.2f.' % (p - 2*rmse, p + 2*rmse))

#### Como encontrar os "melhores" $a$ e $b$?
** => Metodo dos minimos quadrados **

In English: Least Squares Method.<br>
Como seaborn encontra a linha acima?

In [ ]:
from IPython.display import Image
url = 'http://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Linear_least_squares_example2.svg/220px-Linear_least_squares_example2.svg.png'
Image(url)

Pergunta: qual a melhor linha azul que representa o conjunto de pontos vermelhos? <br>
Resposta: a que minimiza a soma dos quadrados das linhas verdes (o erro)

\begin{equation*}
MSE\quad = \frac { 1 }{ n } \sum _{ i=0 }^{ n-1 }{ { { (\hat { { y }^{ (i) } }  } }-{ y }^{ (i) })^{ 2 } }   \quad 
\end{equation*}

\begin{equation*}
RMSE\quad = \sqrt { \frac { 1 }{ n } \sum _{ i\quad =\quad 0 }^{ n-1 }{ { { (\hat { { y }^{ (i) } }  } }-{ y }^{ (i) })^{ 2 } }   } \quad  
\end{equation*}

#### agora com multiplas variáveis

In [ ]:
# primeira observação:
boston_df.iloc[0]

Notação:

$x^{(0)}_{CRIM} = 0.00632$

$x^{(0)}_{ZN} = 18.00000$

$x^{(0)}_{LSTAT} = 4.98$

$y^{(0)} = 24$

A principal "jogada" da regressão linear é considerar que cada feature contribui linearmente na composição do preço:

$a_{CRIM}.x^{(i)}_{CRIM} + a_{ZN}.x^{(i)}_{ZN} + ... + a_{LSTAT}.x^{(i)}_{LSTAT} + b = y^{(i)}$, <br>

para $i = 0, 1, 2,..., n-1$ 

e o objetivo será encontrar $a_{CRIM}, a_{ZN}, ...,a_{LSTAT}, b$ que minimizam o erro 

Numa forma matricial, podemos re-escrever o problema da seguinte maneira:


\begin{equation*}
\mathbf{X}.\mathbf{a} = \mathbf{y}
\end{equation*}

<br>


\begin{equation*}
\mathbf{X} =  \begin{bmatrix}
x^{(0)}_{CRIM} & x^{(0)}_{ZN} & ... & x^{(0)}_{LSTAT} & 1 \\
x^{(1)}_{CRIM} & x^{(1)}_{ZN} & ... & x^{(1)}_{LSTAT} & 1 \\
... & ... & ... & ...\\
x^{(n-1)}_{CRIM} & x^{(n-1)}_{ZN} & ... & x^{(n-1)}_{LSTAT} & 1 \\
\end{bmatrix}
\end{equation*}

<br>

\begin{equation*}
\mathbf{a} =  \begin{bmatrix}
a_{CRIM} \\
a_{ZN} \\
... \\
a_{LSTAT}\\
b \\
\end{bmatrix}
\end{equation*}

<br>

\begin{equation*}
\mathbf{y} =  \begin{bmatrix}
y^{(0)} \\
y^{(1} \\
... \\
y^{(n-1)}\\
\end{bmatrix}
\end{equation*}

#### Engenharia e seleção de features... 

In [ ]:
sns.pairplot(data=boston_df.iloc[:,:4])

In [ ]:
boston_df.columns

### treinamento e predição com sklearn começam aqui...

In [ ]:
# Regressão linear - sklearn
import sklearn
from sklearn.linear_model import LinearRegression

In [ ]:
lreg = LinearRegression()

Funções utilizadas:

* `lreg.fit()` : para treinar o modelo

* `lreg.predict()` : predição do valor, segundo um modelo treinado

* `lreg.score()` : retorna o coeficiente de determinação (R^2), uma medida de quão bem o modelo captura as observações. 

In [ ]:
# Separando as matrizes X (features) e y (labels)

X = boston_df.drop('Preco', axis = 1)
y = boston_df.Preco

In [ ]:
lreg.fit(X, y)

In [ ]:
print ('Valor do coeficiente b tambem chamado de intercept:', lreg.intercept_)

In [ ]:
# Vamos agora ver os coeficientes:
coeff_df = DataFrame(boston_df.columns)
coeff_df.columns = ['Features']

# coluna com os coeficientes
coeff_df["Estimativa dos coeficientes"] = pd.Series(lreg.coef_)

# mostra coeficientes
coeff_df

In [ ]:
coeff_df.set_index('Features').plot(kind = 'bar', figsize = (12, 8))

In [ ]:
# calibrando os coeficientes pelo valor medio da variavel

coeff_df.set_index('Features', inplace = True)

coeff_df = pd.concat([coeff_df, boston_df.mean()], axis = 1).rename(columns = {0: 'media'})

coeff_df['coef_vezes_media'] = coeff_df['Estimativa dos coeficientes'] * coeff_df.media

coeff_df.coef_vezes_media.plot(kind = 'bar', figsize = (12, 8))

In [ ]:
print("Treinei com X: RMSE com y: %.2f"  
      % np.sqrt(np.mean((y - lreg.predict(X)) ** 2)))

## Treinamento e Validação

### Objetivo de separar os dados em treinamento e teste
***
<div class="span5 alert alert-info">

<p> No exemplo acima: </p>
<ul>
  <li> Treinamos e testamos na mesma base </li>
  <li> É esperado que as predições sobre essa base sejam bons, mas e quanto a novos dados? </li>
    <ul>
      <li> sim novos dados</li>
    </ul>
  <li> Um solução seria repartir dados, reservando uma parte para <b>teste</b> e <b>treinando</b> o modelo no restante </li>
  <li> isso se chama validação cruzada </li>  
</ul>
</div>

***

In [ ]:
# Repartindo o dados em treinamento e validação
X_train, X_valid, y_train, y_valid = train_test_split(X,boston_df.Preco)

In [ ]:
# quais são os shapes de cada parte
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

#### Predição de preços

In [ ]:
# recriando o objeto 
lreg = LinearRegression()

# treinando de novo, mas somente com os dados de treinamento
lreg.fit(X_train,y_train)

In [ ]:
# Predição das observações de validação
pred_train = lreg.predict(X_train)
pred_valid = lreg.predict(X_valid)

In [ ]:
print("Treinei com X_train: RMSE com y_train: %.2f"  
      % np.sqrt(np.mean((y_train - pred_train) ** 2)))
    
print("Treinei com X_train, RMSE sobre X_valid e y_valid: %.2f"  
      % np.sqrt( np.mean((y_valid - pred_valid) ** 2)) )

In [ ]:
# R^2 desse fit
lreg.score(X_valid, y_valid)

## Conseguiriamos prever 'atividade por semana'?

a partir de altura, peso, time e IMC??

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('alunos2017.csv')

df['IMC'] = df.Peso/((df.Altura/100) ** 2)

In [ ]:
df.head()

In [ ]:
# del df['Time']

In [ ]:
df['Time'].fillna('ND', inplace=True)

In [ ]:
df.columns

In [ ]:
# X, y = df[[u'Altura', u'Peso', u'IMC', u'Time']], df[u'Atividade_por_semana']
X, y = df[[u'Altura', u'Peso', u'IMC']], df[u'Atividade_por_semana']

In [ ]:
# X = pd.get_dummies(X, columns = ['Time'], drop_first=True)

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [ ]:
lreg = LinearRegression()

In [ ]:
lreg.fit(X_train, y_train)

In [ ]:
pred = pd.Series(lreg.predict(X_valid), index = y_valid.index)
pred

In [ ]:
res_valid = pd.concat([pred, y_valid], axis = 1)

In [ ]:
res_valid.columns = ['pred', 'gold']

In [ ]:
res_valid

In [ ]:
res_valid['erro'] = (res_valid.pred - res_valid.gold) ** 2

In [ ]:
res_valid

In [ ]:
mse = np.mean(res_valid.erro)
print ('MSE =', mse)
print ('RMSE =', np.sqrt(mse))

In [ ]:
X.head()

In [ ]:
lreg.predict([[176, 80, 80.0/(1.76*1.76)]])

In [ ]:
lreg.score(X_valid, y_valid)